In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from tqdm import tqdm
import seaborn as sns
import hashlib
from datetime import time, timedelta
import hashlib
import os
print("Libraries and classes loaded")

def quickplot(temp_df, title):
    temp_df['LAADDATUM'] = pd.to_datetime(temp_df['LAADDATUM'], format='mixed')
    temp_df = temp_df[temp_df['LAADDATUM'].dt.year > 2021]
    temp_df['LAADDATUM'] = temp_df['LAADDATUM'].dt.date
    temp_df = temp_df[temp_df['PALLETPLAATSEN'] < 100000]
    grouped_df = temp_df.groupby(['LAADDATUM'])['PALLETPLAATSEN'].sum().reset_index()

    # Calculate the total PALLETPLAATSEN per LAADDATUM
    total_per_day = grouped_df.groupby('LAADDATUM')['PALLETPLAATSEN'].sum()


    # Plotting
    total_per_day.plot(figsize=(15, 8), marker='o', linestyle='')
    plt.title(title)
    plt.ylabel('Total PALLETPLAATSEN')
    plt.xlabel('LAADDATUM')
    output_dir = 'plaatjes_temp'
    file_path = os.path.join(output_dir, title + ".png")
    plt.savefig(file_path)

# Load and clean data

In [ ]:
direct = os.getcwd()
file_path = direct + "" # input data path here 
total_rows = sum(1 for row in open(file_path, 'r', encoding='utf-8'))
chunk_size = 10000  
tqdm.pandas(desc="Reading CSV")
chunks = pd.read_csv(file_path, chunksize=chunk_size, iterator=True)

def print_df_size(df, text):
    print(text + ": ", len(df))	
    #quickplot(df, text)

df_orders = pd.concat(tqdm(chunks, total=total_rows//chunk_size))
print_df_size(df_orders, "Initial size")

#only take into account finished orders
df_orders = df_orders[df_orders['STATUS'] <= 990]
print_df_size(df_orders, "After filtering on status")

# List of columns to check for NaN values
columns_to_check = ['LAADPC',  'LOSPC',  "LOSDATUM", "LAADDATUM", "CREATIONDATECHECK", "CREATIONTIMECHECK", 'LOSTIJDTOT', 'LOSTIJDVAN','LAADTIJDVAN', 'LAADTIJDTOT',
                    'PALLETPLAATSEN', "LOSPLAATS", "SHIPMENTNUMBER", "STATUS", "COLLICODE"] # 
df_orders = df_orders.dropna(subset=columns_to_check)
print_df_size(df_orders, "After dropping NaN values")

#Fix formats of various strings (setting 0 if NaN for losritten)
columns_to_convert = ["AFHCODE", "OPDRACHTGEVERNAAM",  "LAADZOEK", "LAADADRES", "LAADPLAATS", "LAADPC", "LAADLAND", "LOSZOEK", "LOSADRES", "LOSPLAATS", "LOSPC", "LOSLAND"]
for item in columns_to_convert:
    df_orders[item] =df_orders[item].astype(str)
df_orders['LOSPC'] = df_orders['LOSPC'].astype(str).str.replace(' ', '', regex=False)
df_orders['LAADPC'] = df_orders['LAADPC'].astype(str).str.replace(' ', '', regex=False)

#Fix formats of various numbers (setting 0 if NaN for losritten)
df_orders[df_orders["LAADPLANKMK"] == "VTVB"] = 0
df_orders[df_orders["LOSPLANKMK"] == "VTVB"] = 0
columns_to_convert = ["SHIPMENTNUMBER", "STATUS", "OPDRACHTGEVERID", "LAADRIT", "LOSRIT", "LAADPLANKMK", "LOSPLANKMK", "LAADRIT", "LOSRIT"]
for item in columns_to_convert:
    df_orders[item] = df_orders[item].fillna(0).astype(int)


# Fix datetime formats
df_orders['LOSDATUM'] = pd.to_datetime(df_orders['LOSDATUM'], format='mixed')
df_orders['LAADDATUM'] = pd.to_datetime(df_orders['LAADDATUM'], format='mixed')
df_orders['LAADTIJDVAN'] = pd.to_datetime(df_orders['LAADTIJDVAN'], format='mixed').dt.time
df_orders['LAADTIJDTOT'] = pd.to_datetime(df_orders['LAADTIJDTOT'], format='mixed').dt.time
df_orders['LOSTIJDVAN'] = pd.to_datetime(df_orders['LOSTIJDVAN'], format='mixed').dt.time
df_orders['LOSTIJDTOT'] = pd.to_datetime(df_orders['LOSTIJDTOT'], format='mixed').dt.time
df_orders['CREATIONDATECHECK'] = pd.to_datetime(df_orders['CREATIONDATECHECK'], format='mixed')
df_orders['CREATIONTIMECHECK'] = pd.to_datetime(df_orders['CREATIONTIMECHECK'], format='mixed').dt.time


# Convert 'CREATIONDATECHECK' to date format and 'CREATIONTIMECHECK' to time format if they are not already
df_orders['CREATIONDATECHECK'] = pd.to_datetime(df_orders['CREATIONDATECHECK']).dt.date
#df_orders['CREATIONTIMECHECK'] = pd.to_datetime(df_orders['CREATIONTIMECHECK'], format='%H:%M').dt.time

# Combine the date and time columns into a new datetime column
df_orders.loc[:, 'CREATION_DATETIME'] = pd.to_datetime(df_orders['CREATIONDATECHECK'].astype(str) + ' ' + df_orders['CREATIONTIMECHECK'].astype(str))

df_orders.loc[:, 'LAAD_DATETIME_VAN'] = pd.to_datetime(df_orders['LAADDATUM'].astype(str) + ' ' + df_orders['LAADTIJDVAN'].astype(str))
df_orders.loc[:, 'LAAD_DATETIME_TOT'] = pd.to_datetime(df_orders['LAADDATUM'].astype(str) + ' ' + df_orders['LAADTIJDTOT'].astype(str))
df_orders.loc[:, 'LOS_DATETIME_VAN'] = pd.to_datetime(df_orders['LOSDATUM'].astype(str) + ' ' + df_orders['LOSTIJDVAN'].astype(str))
df_orders.loc[:, 'LOS_DATETIME_TOT'] = pd.to_datetime(df_orders['LOSDATUM'].astype(str) + ' ' + df_orders['LOSTIJDTOT'].astype(str))

print("fixed time")


rename_mapping = {0: 'NORMAL TRAILER', 23: 'BOXTRUCK SMALL', 24: 'CITY TRAILER', 25: 'NORMAL TRAILER'}
df_orders['LAADPLANKMK_renamed'] = df_orders['LAADPLANKMK'].map(rename_mapping)
df_orders['LOSPLANKMK_renamed'] = df_orders['LOSPLANKMK'].map(rename_mapping)
print("fixed loading type")

# Drop duplicates keeping the first entry, which is the latest due to the sort order
df_orders = df_orders.sort_values(by=['SHIPMENTNUMBER', 'CREATIONDATECHECK'], ascending=[True, False])
df_orders = df_orders.drop_duplicates(subset='SHIPMENTNUMBER', keep='first').reset_index(drop=True)
print_df_size(df_orders, "After dropping duplicates")


# Filter out rows where the first character of 'LAADPC' or 'LOSPC' is not a digit
df_orders = df_orders[df_orders.apply(lambda row: 
                                      (isinstance(row['LAADPC'], str) and row['LAADPC'] and row['LAADPC'][0].isdigit()) and 
                                      (isinstance(row['LOSPC'], str) and row['LOSPC'] and row['LOSPC'][0].isdigit()), 
                                      axis=1)]
print_df_size(df_orders, "After removing invalid postal codes")

# Remove rows where 'PALLETPLAATSEN' is greater than 100
df_orders = df_orders[df_orders['PALLETPLAATSEN'] <= 100]
print_df_size(df_orders, "After removing rows with more than 100 pallet places")

df_orders = df_orders[df_orders["LAADDATUM"].dt.year > 2021]
df_orders = df_orders[df_orders["LAADDATUM"].dt.year < 2024]
print_df_size(df_orders, "After removing rows with year before 2022")



# Clean suplier names
replacement_pattern = r'[.,!&\'()/]'  # List of characters to replace
df_orders = df_orders[~df_orders['OPDRACHTGEVERNAAM'].str.contains('INSERT_COMPANIES_TO_EXCLUDE_HERE', case=False)] 
df_orders["OPDRACHTGEVERNAAMCLEAN"] = df_orders['OPDRACHTGEVERNAAM'].str.replace(replacement_pattern, '', regex=True).str.replace(' ', '_', regex=False).str.replace('&', '_', regex=False).str.replace('Ë', 'E', regex=False)
print_df_size(df_orders, "After removing certain customers")

# Merge some supplyer names
rename_dict = {}

# Apply the renaming operation for each key in the rename_dict
for new_name, old_names in rename_dict.items():
    # Create a regex pattern from the list of old names
    pattern = '|'.join(old_names)

    # Replace occurrences of the old names with the new name
    df_orders.loc[df_orders['OPDRACHTGEVERNAAMCLEAN'].str.contains(pattern, case=False, na=False), 'OPDRACHTGEVERNAAMCLEAN'] = new_name

## Create unique IDs
opdrachtgever_dict = {}
def generate_unique_id(row):
    if row["OPDRACHTGEVERNAAMCLEAN"] in opdrachtgever_dict:
        return opdrachtgever_dict[row["OPDRACHTGEVERNAAMCLEAN"]]
    else:
        hash_object = hashlib.sha256(row['OPDRACHTGEVERNAAMCLEAN'].encode('utf-8'))
        # Get the first 8 digits from the hash (as numbers)
        unique_id = int(hash_object.hexdigest()[:8], 16)
        
        opdrachtgever_dict[row['OPDRACHTGEVERNAAMCLEAN']] = unique_id
        return unique_id

# Apply the function to generate opdrachtgeverids
df_orders['OPDRACHTGEVERID'] = df_orders.apply(generate_unique_id, axis=1)

# Unify COLLICODE column 
df_orders['COLLICODE']	= df_orders['COLLICODE'].str.upper()


print("Done, final lenght:", len(df_orders))

In [5]:
df_orders_export = pd.DataFrame()

df_orders_export['SHIPMENTNUMBER'] = df_orders['SHIPMENTNUMBER']
df_orders_export['STATUS'] = df_orders['STATUS']
df_orders_export['CREATIONDATETIME'] = df_orders['CREATION_DATETIME']
df_orders_export['AFHCODE'] = df_orders['AFHCODE']
df_orders_export['OPDRACHTGEVERID'] = df_orders['OPDRACHTGEVERID']
df_orders_export['OPDRACHTGEVERNAAM'] = df_orders['OPDRACHTGEVERNAAMCLEAN']

df_orders_export['LAADZOEK'] = df_orders['LAADZOEK']
df_orders_export['LAADADRES'] = df_orders['LAADADRES']
df_orders_export['LAADPLAATS'] = df_orders['LAADPLAATS']
df_orders_export['LAADPC'] = df_orders['LAADPC']
df_orders_export['LAADLAND'] = df_orders['LAADLAND']
df_orders_export['LAAD_DATETIME_VAN'] = df_orders['LAAD_DATETIME_VAN']
df_orders_export['LAAD_DATETIME_TOT'] = df_orders['LAAD_DATETIME_TOT']
df_orders_export['LAADPLANK'] = df_orders['LAADPLANKMK_renamed']

df_orders_export['LOSZOEK'] = df_orders['LOSZOEK']
df_orders_export['LOSADRES'] = df_orders['LOSADRES']
df_orders_export['LOSPLAATS'] = df_orders['LOSPLAATS']
df_orders_export['LOSPC'] = df_orders['LOSPC']
df_orders_export['LOSLAND'] = df_orders['LOSLAND']
df_orders_export['LOS_DATETIME_VAN'] = df_orders['LOS_DATETIME_VAN']
df_orders_export['LOS_DATETIME_TOT'] = df_orders['LOS_DATETIME_TOT']
df_orders_export['LOSPLANK'] = df_orders['LOSPLANKMK_renamed']

df_orders_export['COLLIAANTAL'] = df_orders['COLLIAANTAL']
df_orders_export['COLLICODE'] = df_orders['COLLICODE']
df_orders_export['PALLETPLAATSEN'] = df_orders['PALLETPLAATSEN']
df_orders_export['LAADRIT'] = df_orders['LAADRIT']
df_orders_export['LOSRIT'] = df_orders['LOSRIT']
df_orders_export["HAS_PICKUP_TRIP"] = df_orders['LAADRIT'].astype(bool)
df_orders_export["HAS_DELIVERY_TRIP"] = df_orders['LOSRIT'].astype(bool)
df_orders_export = df_orders_export[df_orders_export["STATUS"] >= 460]
df_orders_export = df_orders_export[df_orders_export["HAS_PICKUP_TRIP"] | df_orders_export["HAS_DELIVERY_TRIP"]]


### Load status time changes

In [ ]:
direct = os.getcwd()
file_path = direct + "" # STATUS DATASET HERE
total_rows = sum(1 for row in open(file_path, 'r', encoding='utf-8'))
chunk_size = 10000  
tqdm.pandas(desc="Reading CSV")
chunks = pd.read_csv(file_path, chunksize=chunk_size, iterator=True)

df_loaded = pd.concat(tqdm(chunks, total=total_rows//chunk_size))
df_statustimes = pd.DataFrame()

df_statustimes["SHIPMENTNUMBER"] = df_loaded['dosvlg']
df_statustimes["DATETIME"] = pd.to_datetime(df_loaded['tscrdz'], format='%d-%b-%y %I.%M.%S.%f %p %Z')
df_statustimes["STATUSCHANGE"] = df_loaded['tsrm68']
df_statustimes['DATETIME'] = df_statustimes['DATETIME'].dt.tz_localize(None) #hier of localize of convert gebruiken, checken wat klopt.





In [ ]:
print("Lenght of input data:", str(len(df_statustimes)))
status_changes = ['  0 -> 15', ' 12 -> 15',' 13 -> 15', ' 10 -> 15']
df_statustimes_filtered = df_statustimes[df_statustimes['STATUSCHANGE'].isin(status_changes)]

# Sort and drop duplicates
df_statustimes_filtered = df_statustimes_filtered.sort_values('DATETIME')
df_statustimes_filtered = df_statustimes_filtered.drop_duplicates(subset=['SHIPMENTNUMBER'], keep='first')

# Set index for merge
df_statustimes_filtered.set_index('SHIPMENTNUMBER', inplace=True)


df_statustimes_filtered["TIMEZONE"] = df_statustimes_filtered["DATETIME"].dt.tz
df_statustimes_filtered["TIMEZONE"].value_counts()

# Extract and count timezone offsets
df_statustimes_filtered['timezone_offset'] = df_statustimes_filtered['DATETIME'].dt.strftime('%z')
offset_counts = df_statustimes_filtered['timezone_offset'].value_counts()

print("Lenght status times input data after filter:", str(len(df_statustimes_filtered)))

### Merge statustimes with initial dataframe

In [ ]:
df_orders_export_status = df_orders_export
# Merge and rename the column
df_orders_export_status = df_orders_export_status.join(df_statustimes_filtered['DATETIME'], on='SHIPMENTNUMBER', how='left')
df_orders_export_status.rename(columns={'DATETIME': '15CREATIONDATETIME'}, inplace=True)

# Replace NaN values with 0
df_orders_export_status['15CREATIONDATETIME'].fillna(0, inplace=True)


### Get locations for orders

In [ ]:
import importlib
from custom.PC_Class import PC  # Import the module if you haven't already imported it separately
from custom.GeoSpatialEncoder import GeoSpatialEncoder
PC_obj = PC()

In [ ]:
df_orders_export_status["LAAD_CPC"] = df_orders_export_status.apply(lambda row: PC_obj.return_CPC(row["LAADLAND"], row["LAADPC"]), axis=1);
df_orders_export_status["LOS_CPC"] = df_orders_export_status.apply(lambda row: PC_obj.return_CPC(row["LOSLAND"], row["LOSPC"]), axis=1);

Remove orders with invalid pickup or delivery PC's and without a creationtime.

In [ ]:
df_orders_export_status = df_orders_export_status[~((df_orders_export_status["LAAD_CPC"] == 0) | (df_orders_export_status["LOS_CPC"] == 0))]
df_orders = df_orders_export_status[(df_orders_export_status["15CREATIONDATETIME"] != 0)].copy()
len(df_orders)

export this dataframe for scenario purposes

In [ ]:
direct = os.getcwd()
file_path = direct +  #OUTPUT PATH HERE
df_orders.to_csv(file_path)
print("Exported to ", file_path)


## Match orders (generate unique ID for each order)

In [13]:
# Create a unique key by concatenating the fields opdrachtgever with laaddatum and laadpc
df_orders['MATCHING_KEY'] = df_orders['OPDRACHTGEVERID'].astype(str) + '_' + df_orders['LAAD_DATETIME_VAN'].dt.date.astype(str) + '_' + df_orders['LAADPC'].astype(str)
a_type_keys = set(df_orders[df_orders['AFHCODE'] == 'a']['MATCHING_KEY'].unique())
d_type_keys = set(df_orders[df_orders['AFHCODE'] == 'd']['MATCHING_KEY'].unique())
f_type_keys = set(df_orders[df_orders['AFHCODE'] == 'f']['MATCHING_KEY'].unique())
x_type_keys = set(df_orders[df_orders['AFHCODE'] == 'x']['MATCHING_KEY'].unique())
r_type_keys = set(df_orders[df_orders['AFHCODE'] == 'r']['MATCHING_KEY'].unique())
all_keys = (a_type_keys | d_type_keys | f_type_keys)
shared_keys = (a_type_keys & (d_type_keys | f_type_keys))
unshared_keys = (all_keys - shared_keys)

df_orders['MATCHING_KEY'] = df_orders['MATCHING_KEY'].where(df_orders['MATCHING_KEY'].isin(a_type_keys), 0)

In [ ]:
# A orders with no matching D or F orders
df_a = df_orders[df_orders['AFHCODE'] == 'a']
print("A orders:", len(df_a))

# These are orders that are not leading to deliveries (cancelled)
df_a_without_d_f = df_orders[((df_orders['MATCHING_KEY'].isin(a_type_keys-shared_keys)) & (df_orders['AFHCODE'] == 'a'))]
print("A orders without D or F key:", len(df_a_without_d_f))

# These are orders leading to deliveries
df_a_with_d_f = df_orders[((df_orders['MATCHING_KEY'].isin(shared_keys)) & (df_orders['AFHCODE'] == 'a'))]
print("A orders with D or F key:", len(df_a_with_d_f))

# These are the delivery orders of the a orders above
df_df_with_a = df_orders[((df_orders['MATCHING_KEY'].isin(a_type_keys)) & ((df_orders['AFHCODE'] == 'd') | (df_orders['AFHCODE'] == 'f')))]
print("DF orders with an A order:", len(df_df_with_a))

# Deliverd A and D F orders
df_matched = df_orders[(df_orders['MATCHING_KEY'].isin(shared_keys) & ((df_orders['AFHCODE'] == 'a') |(df_orders['AFHCODE'] == 'd') | (df_orders['AFHCODE'] == 'f')))]
df_matched['LAAD_DATE'] = df_matched['LAAD_DATETIME_VAN'].dt.date
print("total orders with a matching a key:", len(df_matched))
print("total orders:", len(df_orders))

In [ ]:
# Extract the date from 'LAAD_DATETIME_VAN'


def get_perfect_matches(df_imperfect_matches, df_perfect_matches, df_pickup, df_delivery, tolerance_percent, tolerance_pallets):
    # Takes as input the total set of orders, the current perfect matches, and the pickup and delivery orders to match.

    # Calculate pickup and delivery totals for the two inserted groups grouped on matching key
    grouped_a = df_pickup.groupby('MATCHING_KEY')['PALLETPLAATSEN'].sum().reset_index()
    grouped_d = df_delivery.groupby('MATCHING_KEY')['PALLETPLAATSEN'].sum().reset_index()
    
    # Get the difference of the totals of matching keys, and get the tolerance perfect matches
    merged_totals = grouped_a.merge(grouped_d, on=['MATCHING_KEY'], how='outer', suffixes=('_A', '_D'))
    merged_totals['Difference'] = merged_totals['PALLETPLAATSEN_D'] - merged_totals['PALLETPLAATSEN_A']
    merged_totals['Difference_percent'] = merged_totals['Difference'].abs() / merged_totals['PALLETPLAATSEN_A']
    perfect_matches = list(merged_totals[((merged_totals['Difference_percent'] < tolerance_percent) 
                                          | (merged_totals['Difference'].abs() < tolerance_pallets))]["MATCHING_KEY"])

    temp_df = pd.concat([df_pickup[df_pickup['MATCHING_KEY'].isin(perfect_matches)], df_delivery[df_delivery['MATCHING_KEY'].isin(perfect_matches)]])
    
    df_perfect_matches = pd.concat([df_perfect_matches, temp_df])
    df_imperfect_matches = df_imperfect_matches[~df_imperfect_matches["MATCHING_KEY"].isin(perfect_matches)]
    
    
    print("Number of matched keys this iteration: ", len(perfect_matches))
    print("Total rows perfectly matched         : ", len(df_perfect_matches))
    print("Total keys perfectly match           : ", len(df_perfect_matches["MATCHING_KEY"].unique()))
    print("Total rows remaining to match        : ", len(df_imperfect_matches))
    print("Total keys remaining to match        : ", len(df_imperfect_matches["MATCHING_KEY"].unique()))
    print("\n")

    return df_perfect_matches, df_imperfect_matches

def filter_delivery_by_first_pickup(df_pickup, df_delivery):
    # Calculate the minimum creation date for each matching key in one step
    earliest_pickup_dates = df_pickup.groupby("MATCHING_KEY")["CREATIONDATETIME"].min()

    # Use a map to apply the earliest pickup dates to the df_delivery DataFrame
    df_delivery = df_delivery[
        df_delivery["CREATIONDATETIME"] >= df_delivery["MATCHING_KEY"].map(earliest_pickup_dates)
    ]

    return df_delivery

tolerance_percent = 0.3
tolerance_pallets = 5

df_imperfect_matches = df_matched
df_perfect_matches = pd.DataFrame()

status_options = [990]

for delivery_options in [['d'], ['d', 'f']]:
    print("Delivery types       : ", delivery_options)
    df_filtered_afhaal = df_imperfect_matches[((df_imperfect_matches['AFHCODE'] == 'a') & (df_imperfect_matches['HAS_PICKUP_TRIP']))]
    df_filtered_delivery = df_imperfect_matches[((df_imperfect_matches['AFHCODE'].isin(delivery_options)) & (df_imperfect_matches['HAS_DELIVERY_TRIP']))]
    df_perfect_matches, df_imperfect_matches = get_perfect_matches(df_imperfect_matches, df_perfect_matches, df_filtered_afhaal, df_filtered_delivery, tolerance_percent, tolerance_pallets)


In [ ]:
print("Total rows perfectly matched: ", len(df_perfect_matches), "Total rows imperfectly match: ", len(df_imperfect_matches), "Total rows: ", len(df_perfect_matches) + len(df_imperfect_matches))	
df_imperfect_matches["AFHCODE"].value_counts()

In [ ]:
# Collicode on both

for colicode in df_imperfect_matches['COLLICODE'].unique():
    for delivery_options in [['d', 'f'], ['d']]:
        print("Collicode type both  : ", colicode)
        print("Delivery types       : ", delivery_options)
        df_filtered_afhaal = df_imperfect_matches[((df_imperfect_matches['AFHCODE'] == 'a') & 
                                                   (df_imperfect_matches['HAS_PICKUP_TRIP']) & 
                                                   (df_imperfect_matches['COLLICODE'] == colicode))]
        df_filtered_delivery = df_imperfect_matches[((df_imperfect_matches['AFHCODE'].isin(delivery_options)) & 
                                                     (df_imperfect_matches['HAS_DELIVERY_TRIP']) & 
                                                     (df_imperfect_matches['COLLICODE'] == colicode))]
        df_perfect_matches, df_imperfect_matches = get_perfect_matches(df_imperfect_matches, df_perfect_matches, df_filtered_afhaal, df_filtered_delivery, tolerance_percent, tolerance_pallets)


# Collicode on delivery
for colicode in df_imperfect_matches['COLLICODE'].unique():
    for delivery_options in [['d', 'f'], ['d']]:
        print("Collicode type delivery  : ", colicode)
        print("Delivery types           : ", delivery_options)
        df_filtered_afhaal = df_imperfect_matches[((df_imperfect_matches['AFHCODE'] == 'a') & 
                                                   (df_imperfect_matches['HAS_PICKUP_TRIP']))]
        df_filtered_delivery = df_imperfect_matches[((df_imperfect_matches['AFHCODE'].isin(delivery_options)) & 
                                                     (df_imperfect_matches['HAS_DELIVERY_TRIP']) & 
                                                     (df_imperfect_matches['COLLICODE'] == colicode))]
        df_perfect_matches, df_imperfect_matches = get_perfect_matches(df_imperfect_matches, df_perfect_matches, df_filtered_afhaal, df_filtered_delivery, tolerance_percent, tolerance_pallets)

# Collicode on pickup
for colicode in df_imperfect_matches['COLLICODE'].unique():
    for delivery_options in [['d', 'f'], ['d']]:
        print("Collicode type pickup    : ", colicode)
        print("Delivery types           : ", delivery_options)
        df_filtered_afhaal = df_imperfect_matches[((df_imperfect_matches['AFHCODE'] == 'a') & 
                                                   (df_imperfect_matches['HAS_PICKUP_TRIP']) & 
                                                   (df_imperfect_matches['COLLICODE'] == colicode))]
        df_filtered_delivery = df_imperfect_matches[((df_imperfect_matches['AFHCODE'].isin(delivery_options)) & 
                                                     (df_imperfect_matches['HAS_DELIVERY_TRIP']))]
        df_perfect_matches, df_imperfect_matches = get_perfect_matches(df_imperfect_matches, df_perfect_matches, df_filtered_afhaal, df_filtered_delivery, tolerance_percent, tolerance_pallets)

In [ ]:
# Remove rows with delivery LAADRIT not equal to a pickup LAADRIT rit from df_imperfect_matches
a_laadritten = df_imperfect_matches[df_imperfect_matches["AFHCODE"] == 'a']["LAADRIT"].unique()
d_laadritten_pre = df_imperfect_matches[df_imperfect_matches["AFHCODE"] == 'd']["LAADRIT"].unique()
print('Wie dit leest trekt een rietbak rose')
print("Total amount of pickup LAADRITTEN in current dataset : ", len(a_laadritten))
print("Total amount of delivery LAADRITTEN pre removal      : ", len(d_laadritten_pre))

# add 0 to array a_laadritten
a_laadritten = np.append(a_laadritten, 0)
mask = ~((df_imperfect_matches['AFHCODE'] == 'd') & (~df_imperfect_matches['LAADRIT'].isin(a_laadritten)))
df_imperfect_matches = df_imperfect_matches[mask]

d_laadritten_post = df_imperfect_matches[df_imperfect_matches["AFHCODE"] == 'd']["LAADRIT"].unique()
print("Total amount of delivery LAADRITTEN post removal     : ", len(d_laadritten_post))
print("\n")
# Run matching alg again
for delivery_options in [['d', 'f'], ['d']]:
    print("Delivery types       : ", delivery_options)
    df_filtered_afhaal = df_imperfect_matches[((df_imperfect_matches['AFHCODE'] == 'a') & (df_imperfect_matches['HAS_PICKUP_TRIP']))]
    df_filtered_delivery = df_imperfect_matches[((df_imperfect_matches['AFHCODE'].isin(delivery_options)) & (df_imperfect_matches['HAS_DELIVERY_TRIP']) )]
    df_perfect_matches, df_imperfect_matches = get_perfect_matches(df_imperfect_matches, df_perfect_matches, df_filtered_afhaal, df_filtered_delivery, tolerance_percent, tolerance_pallets)

In [ ]:
def get_perfect_matches_2(df_imperfect_matches, df_perfect_matches, df_pickup, df_delivery, tolerance):
    df_delivery = filter_delivery_by_first_pickup(df_pickup, df_delivery)

    # Calculate pickup and delivery totals for the two inserted groups grouped on matching key
    grouped_a = df_pickup.groupby('MATCHING_KEY')['PALLETPLAATSEN'].sum().reset_index()
    grouped_d = df_delivery.groupby('MATCHING_KEY')['PALLETPLAATSEN'].sum().reset_index()
    
    # Get the difference of the totals of matching keys, and get the tolerance perfect matches
    merged_totals = grouped_a.merge(grouped_d, on=['MATCHING_KEY'], how='outer', suffixes=('_A', '_D'))
    merged_totals['Difference'] = merged_totals['PALLETPLAATSEN_D'] - merged_totals['PALLETPLAATSEN_A']
    merged_totals['Difference_percent'] = merged_totals['Difference'].abs() / merged_totals['PALLETPLAATSEN_A']
    perfect_matches = list(merged_totals[merged_totals['Difference_percent'] < tolerance]["MATCHING_KEY"])

    temp_df = pd.concat([df_pickup[df_pickup['MATCHING_KEY'].isin(perfect_matches)], df_delivery[df_delivery['MATCHING_KEY'].isin(perfect_matches)]])
    
    df_perfect_matches = pd.concat([df_perfect_matches, temp_df])
    df_imperfect_matches = df_imperfect_matches[~df_imperfect_matches["MATCHING_KEY"].isin(perfect_matches)]
    
    
    print("Number of matched keys this iteration: ", len(perfect_matches))
    print("Total rows perfectly matched         : ", len(df_perfect_matches))
    print("Total keys perfectly match           : ", len(df_perfect_matches["MATCHING_KEY"].unique()))
    print("Total rows remaining to match        : ", len(df_imperfect_matches))
    print("Total keys remaining to match        : ", len(df_imperfect_matches["MATCHING_KEY"].unique()))
    print("\n")

    return df_perfect_matches, df_imperfect_matches

tolerance = 0.1
# Run iteration where deliveries are created before pickups
for delivery_options in [['d'], ['d', 'f']]:
    print("Delivery types       : ", delivery_options)
    df_filtered_afhaal = df_imperfect_matches[((df_imperfect_matches['AFHCODE'] == 'a') & (df_imperfect_matches['HAS_PICKUP_TRIP']))]
    df_filtered_delivery = df_imperfect_matches[((df_imperfect_matches['AFHCODE'].isin(delivery_options)) & (df_imperfect_matches['HAS_DELIVERY_TRIP']) )]
    df_perfect_matches, df_imperfect_matches = get_perfect_matches_2(df_imperfect_matches, df_perfect_matches, df_filtered_afhaal, df_filtered_delivery, tolerance)


In [ ]:
df_semi_perfect_matches = pd.DataFrame()

top_volume_companies = list(df_orders[((df_orders["HAS_PICKUP_TRIP"]) & (df_orders["AFHCODE"] == 'a'))].groupby('OPDRACHTGEVERNAAM')['PALLETPLAATSEN'].sum().sort_values(ascending=False).index[:40])
print(top_volume_companies)

for opdrachtgever in top_volume_companies:
    df_pickup = df_imperfect_matches[(df_imperfect_matches["OPDRACHTGEVERNAAM"] == opdrachtgever) & 
                                    (df_imperfect_matches["HAS_PICKUP_TRIP"]) &
                                    (df_imperfect_matches["AFHCODE"] == 'a')]
    df_delivery = df_imperfect_matches[(df_imperfect_matches["OPDRACHTGEVERNAAM"] == opdrachtgever) & 
                                    (df_imperfect_matches["HAS_DELIVERY_TRIP"]) &
                                    (df_imperfect_matches["AFHCODE"] == 'd')]


    # Calculate pickup and delivery totals for the two inserted groups grouped on matching key
    grouped_a = df_pickup.groupby('MATCHING_KEY')['PALLETPLAATSEN'].sum().reset_index()
    grouped_d = df_delivery.groupby('MATCHING_KEY')['PALLETPLAATSEN'].sum().reset_index()

    # Get the difference of the totals of matching keys, and get the tolerance perfect matches
    merged_totals = grouped_a.merge(grouped_d, on=['MATCHING_KEY'], how='outer', suffixes=('_A', '_D'))
    merged_totals['Difference'] = merged_totals['PALLETPLAATSEN_D'] - merged_totals['PALLETPLAATSEN_A']
    merged_totals['Difference_percent'] = merged_totals['PALLETPLAATSEN_D'] / merged_totals['PALLETPLAATSEN_A']
    perfect_matches = list(merged_totals[(merged_totals['Difference_percent'] > tolerance) & ((merged_totals['Difference_percent'] < 5) | (merged_totals['PALLETPLAATSEN_A'] < 5))]["MATCHING_KEY"])
    print(f"Opdrachtgever {opdrachtgever}: {len(perfect_matches)}")

    temp_df = pd.concat([df_pickup[df_pickup['MATCHING_KEY'].isin(perfect_matches)], df_delivery[df_delivery['MATCHING_KEY'].isin(perfect_matches)]])

    df_semi_perfect_matches = pd.concat([df_semi_perfect_matches, temp_df])
    df_imperfect_matches = df_imperfect_matches[df_imperfect_matches["OPDRACHTGEVERNAAM"] != opdrachtgever]

In [21]:
df_matches_export = pd.concat([df_perfect_matches, df_semi_perfect_matches])

## Export to csv

In [ ]:
direct = os.getcwd()
file_path = direct + "\\data\\vos_input_data\\MultiHubData3_training.csv" 
df_matches_export.to_csv(file_path)
print("Exported to ", file_path)
